## Import everything needed and create a parameters dictionnary

In [93]:
import os #Fournit un moyen portable d'utiliser certaines fonctionnalites dependantes de Windows
import numpy as np #Utilisee pour le calcul numerique et la manipulation de tableaux multidimensionnels
import pandas as pd #Manipulation et analyse de donnees
import matplotlib.pyplot as plt #Creation de graphiques et de visualisations comme matlab
from matplotlib.patches import Polygon #Creation de graphiques et de visualisations comme matlab
from scipy.ndimage import gaussian_filter as smooth
from own_Functions import *
from median_thomas import *
import gc
from IPython.display import clear_output

#Dictionary to set-up the color palette for the graphs and HTML (turns and phases colors)
turnPalette= {'gogd':'#86d065', #green #Good Object Good Direction
              'gobd':'#f1c232', #yellow #Good Object Bad Direction
              'bogd':'#e69138', #orange #Bad Object Good Direction
              'bobd':'#cc0000', #red #Bad Object Bad Direction
              'gogdet':'#7754e0', #purple #Good Object Good Direction Extra Turn
              'gobdet':'#ef2388', #pink #Good Object Bad Direction Extra Turn
              'all turns':'#49a1f1'} #blue #All turns

phasePalette= {'Phase1':'#b4a7d6', #purple
               'Phase2':'#94ccff', #blue
               'Phase3':'#84d567', #green
               'Phase4':'#f5b779', #orange
               'Other':'#fbabd2'} #pink

#titlesPalette=
#Examples
#'gobd':(241/255,194/255,50/255) --> RVB
#'gobd':'#f1c232' --> Hexadecimal
#Dashed=pointilles / Solid=trait plein

#Intializes the value of some variables and defines some important global variables
REMAINING_REWARDS = False #If true, indicates the number of reward available on an object when the mouse starts to go around

MINIMAL_DURATION_STOP = 0.1 #If a stop is shorter than this, merges the two epochs bordering it
MINIMAL_DURATION_EPOCH = 0.3 #Minimal duration of an epoch to be considered
TRAPEZE_SIZE = 35 #How much to extend outside of the objects to detect the trajectories
Resolution = 512,512 #Variable which stores the size of the experimental field in pixels
true_sigma = 1 #The sigma used for the remaining of the analysis
true_cut_speed = 7 #Value to be used for the remaining of the analysis
true_ecart_angle = 1 #If a change is made, must change timeofframes

#Defines the coordinates of each patch
NWpatch_coords = [[104, Resolution[1] -  125], [173, Resolution[1] -  125], [173, Resolution[1] -  201], [104, Resolution[1] -  201]]
NEpatch_coords = [[330, Resolution[1] -  120], [400, Resolution[1] -  120], [400, Resolution[1] -  200], [330, Resolution[1] -  200]]
SWpatch_coords = [[109, Resolution[1] -  351], [181, Resolution[1] -  351], [181, Resolution[1] -  410], [109, Resolution[1] -  410]]
SEpatch_coords = [[330, Resolution[1] -  350], [400, Resolution[1] -  350], [400, Resolution[1] -  410], [330, Resolution[1] -  410]]

#Defines the list of trapezes as a dictionnary containing, dictionnary to access them more easily
#Dictionary of trapezes
#Dictionnary of the different types of stops
#The value kept in the list are a list of the beginning epoch of the stop and end of the stop

## Select the good animal

In [94]:
#thisOSName=platform.system() #Detects in which computer the code is running 
#If thisOSName=='Darwin': #In case of use in Maud's personal computer
#Folder_path='/Users/davidrobbe/MEGAsync/Python/CorticoStriatalProject/Videos/' #Need to be changed depending the actual path in MY OWN computer 
#elif thisOSName=='Linux': #In case of use in Maud's office computer
#Folder_path='/home/david/MyLocalData/Maud/'
#For when on Alice's computer : 
#Folder_path = '/home/david/Desktop/Kenza/'
Folder_path = 'C:'+os.sep+'Users'+os.sep+'david'+os.sep+'Desktop'+os.sep+'Mouse'+os.sep+'Low certainty'+os.sep

mouseName='MOUKA254_1G2D' #This is the only parameter to change in this cell (in this computer)

mouseFolder_Path=Folder_path+mouseName
codename =  "AnalyzesGeneration_05-2.ipynb"
os.chdir(mouseFolder_Path)
sessionFolders=[ name for name in os.listdir(mouseFolder_Path) if os.path.isdir(os.path.join(mouseFolder_Path, name)) and name[0:3]=='MOU']
sessionFolders.sort()

## Functions
##### All functions used in this code are described here 

In [95]:
#Cell containing the functions from Alice's code
import pickle
def trapezes_from_patch(patch, width):
    """
    generate the trapezes coordinates surrounding a patch
    inputs:
    patch - coordinates of a patch [[Xa, Ya], [Xb, Yb], [Xc, Yc], [Xd, Yd]]
    width - width of the trapeze in pixels
    outputs:
    coordinates [[Xa, Ya], [Xb, Yb], [Xc, Yc], [Xd, Yd]] for the 4 trapezes.
    
    trapezes_from_patch(SWpatch_coords, 200)
    """

    N = [patch[0], patch[1], [patch[1][0]+width, patch[1][1]+width], [patch[0][0]-width, patch[0][1]+width]]
    E = [patch[1], patch[2], [patch[2][0]+width, patch[2][1]-width], [patch[1][0]+width, patch[1][1]+width]]
    S = [patch[2], patch[3], [patch[3][0]-width, patch[3][1]-width], [patch[2][0]+width, patch[2][1]-width]]
    W = [patch[3], patch[0], [patch[0][0]-width, patch[0][1]+width], [patch[3][0]-width, patch[3][1]-width]]
    return N, E, S, W

collection_trapeze = {"NW" : dict(), "NE" : dict(), "SW" : dict(), "SE" : dict()}
collection_trapeze["NW"]["N"], collection_trapeze["NW"]["E"], collection_trapeze["NW"]["S"], collection_trapeze["NW"]["W"] = trapezes_from_patch(NWpatch_coords, TRAPEZE_SIZE)
collection_trapeze["NE"]["N"], collection_trapeze["NE"]["E"], collection_trapeze["NE"]["S"], collection_trapeze["NE"]["W"] = trapezes_from_patch(NEpatch_coords, TRAPEZE_SIZE)
collection_trapeze["SW"]["N"], collection_trapeze["SW"]["E"], collection_trapeze["SW"]["S"], collection_trapeze["SW"]["W"] = trapezes_from_patch(SWpatch_coords, TRAPEZE_SIZE)
collection_trapeze["SE"]["N"], collection_trapeze["SE"]["E"], collection_trapeze["SE"]["S"], collection_trapeze["SE"]["W"] = trapezes_from_patch(SEpatch_coords, TRAPEZE_SIZE)

def search_right_turn(time_start, time_end, dinfo):
    for i in range(len(dinfo)) :
        if dinfo.iat[i, 0] > time_start and dinfo.iat[i, 0] < time_end : #If the time of the turn is comprised between the beginning and the end of the epoch
            return dinfo.iat[i, 12] #Return the max number of rewards of the serie
    return -1

#From now to the end of the cell, this function is used to do the analysis
def AnalysisFunction(session, mouseFolder_Path):
    stops_type = {"rewarded" : [],
              "unrewarded" : []}

    try :
    #Gets the parameters of the session
        session_param = pd.read_csv(mouseFolder_Path + '/' + session + '/' + session + "_sessionparam.csv")
        session_direction = session_param['potentialRewardedDirections'][0]

        #Gets the positional informations and filters the dataframe to keep only the relevant informations
        csvCentroid_fullpath=mouseFolder_Path + '/' + session + '/' + session + '_centroidTXY.csv'
        df=pd.read_csv(csvCentroid_fullpath) #Transforms CSV file into panda dataframe
        df=df.dropna() #Deletes lines with one or more NA
        df=df.loc[df['time'] >15] #First seconds of the video contained artefacts so we need to delete them
        df=df[df['xposition'].between(1,500) & df['yposition'].between(1,500)] #The values between 15 and 500 are kept (le tableau est cree plus grand que necessaire)

        csvTurnsinfo_fullpath=mouseFolder_Path + '/' + session + '/' + session + '_turnsinfo.csv'#Gets the information on the turns in the dataframe dinfo
        dinfo=pd.read_csv(csvTurnsinfo_fullpath) #Transforms CSV file into panda dataframe
        for i in range( dinfo.index.values[-1]):#If there is a missing value for ongoingRewardedObject, replace it with either SW or SE, as long as it's not the one where the mouse is
            if (type(dinfo['ongoingRewardedObject'][i]) == float) :
                dinfo.iat[i, 8] = str([dinfo.iat[i, 4]]) 

        dinfo=dinfo.loc[dinfo['time'] >15] #Il y a des artefacts sur les premieres secondes de videos, donc il faut les supprimer

    except : 
        return -1 #Signals that there was an error

    size_of_df = len(df) #Gets some informations in a more convenient form
    time = df['time'].to_numpy()
    xposition = df['xposition'].to_numpy()
    yposition = df['yposition'].to_numpy()
    yposition = Resolution[1] - yposition #y position is inverted, puts it back in the right way
    xgauss = smooth(xposition, true_sigma) ; ygauss = smooth(yposition, true_sigma) #Smoothes the positions with true sigma
    
    #Does the actual analysis. The remaining part consists in accessing the pertinent informations and plotting them
    distance, speed, time_average, acceleration, angles, angular_speed, list_epochs = analysis_trajectory(
        time, xgauss, ygauss, collection_trapeze, dinfo, true_cut_speed, true_ecart_angle, Resolution,
        MIN_DURATION_EPOCH= MINIMAL_DURATION_EPOCH, MIN_DURATION_STOP= MINIMAL_DURATION_STOP)
    
    #Prepares lists of epochs corresponding to diffrent types of behavior
    for i in range(len(list_epochs) - 1) :
        if list_epochs[i][2][0] == "Q" : #If this is a quarter turn
            if list_epochs[i][2][2] == "G" : #If this is a good turn and thus a rewarded quarter turn
                stops_type["rewarded"].append([list_epochs[i][1], list_epochs[i + 1][0]])
            else : #Then the quarter turn was not rewarded
                stops_type["unrewarded"].append([list_epochs[i][1], list_epochs[i + 1][0]])
        elif list_epochs[i][2][0] == "B" : #If this is a between objects
            if list_epochs[i][2][5] == 'r' : #If the trajectory was rewarded
                stops_type["rewarded"].append([list_epochs[i][1], list_epochs[i + 1][0]])
            else : #Then the between objects was unrewarded
                stops_type["unrewarded"].append([list_epochs[i][1], list_epochs[i + 1][0]])

    list_quarter_turn = [epoch for epoch in list_epochs if epoch[2][0] == "Q"] #Gets the list of all quarter turns
    list_between_objects = [epoch for epoch in list_epochs if epoch[2][0] == "B"] #Gets all trajectories between objects
    list_toward_object = [epoch for epoch in list_epochs if epoch[2][0] == "T"] #Gets all trajectories towards objects
    list_movement_not_quarter = [epoch for epoch in list_epochs if epoch[2][0] == "N"] #Gets all explorative trajectories
    list_of_stops = [[list_epochs[a - 1][1 ] + 1, list_epochs[a][0] - 1] for a in range(1, len(list_epochs))]

    #Creates a list for each type of quarter turn
    rewarded = [epoch for epoch in list_quarter_turn if epoch[2][2] == 'G']
    unrewarded = [epoch for epoch in list_quarter_turn if epoch[2][2] != 'G']
    extra = [epoch for epoch in list_quarter_turn if epoch[2][2] == 'E']
    badDirection = [epoch for epoch in list_quarter_turn if epoch[2][2] == 'B']
    wrongObject = [epoch for epoch in list_quarter_turn if epoch[2][2] == 'O']
    doubleWrong = [epoch for epoch in list_quarter_turn if epoch[2][2] == 'H']

    between_reward = [epoch for epoch in list_between_objects if epoch[2][5] == 'r']
    between_unrewarded = [epoch for epoch in list_between_objects if epoch[2][5] == 'n']

    anti_clock_turn = [epoch for epoch in list_quarter_turn if epoch[2][1] == "k"]
    clock_turn = [epoch for epoch in list_quarter_turn if epoch[2][1] == "w"]
    exploring = [epoch for epoch in list_quarter_turn if epoch[2][2] == 'X']

    ############################################
    #Creates pickel for number of rewarded and unrewarded quarters / number of CW and CCW turns
    reward_number=len(rewarded)
    unreward_number=len(unrewarded)
    extra_number=len(extra)
    filename = mouseFolder_Path + '/' + session + '/' + 'pickle_total_reward_unreward_extra.pkl'
    
    pickle.dump((reward_number, unreward_number, extra_number), open(filename, 'wb'))

    CW_turn=len(clock_turn)
    CCW_turn=len(anti_clock_turn)
    filename2 = mouseFolder_Path + '/' + session + '/' + 'pickle_total_CW_CCW.pkl'
    
    pickle.dump((CW_turn, CCW_turn), open(filename2, 'wb'))

    ############################################
    #Creates the graph in the "Behavior" HTML

    list_number_reward = []
    serie = 0 #Will indicate if we are in a serie
    current_quarter_turn = -1

    for epoch in list_epochs :
        if epoch[2][0] == "Q" :
            current_quarter_turn += 1 #Adds the number of the current quarter turn after so it still indicates the last one of the serie when added
            if epoch[2][2] == 'G': #If there is a good quarter turn, enter or continue a serie
                
                if serie == 0 : #Adds the signal at the beginning of the serie
                    list_number_reward.append([current_quarter_turn, search_right_turn(time_start= time[list_quarter_turn[current_quarter_turn][0]], time_end= time[list_quarter_turn[current_quarter_turn][1]], dinfo = dinfo)])
                    serie = 1

                #serie = 1

            elif serie == 1 : #If the epoch is not a rewarded quarter turn
                #list_number_reward.append([current_quarter_turn - 1, search_right_turn(time_start= time[list_quarter_turn[current_quarter_turn][0]], time_end= time[list_quarter_turn[current_quarter_turn][1]], dinfo = dinfo)])
                serie = 0

        elif serie == 1 : #If the epoch is not a rewarded quarter turn
            #list_number_reward.append([current_quarter_turn, search_right_turn(time_start= time[list_quarter_turn[current_quarter_turn][0]], time_end= time[list_quarter_turn[current_quarter_turn][1]], dinfo = dinfo)])
            serie = 0

    fig = plt.figure(figsize=(60, 5))
    plt.plot([time_average[i[0]] if i[2][0] == "Q"  else time_average[i[1]] for i in list_epochs if i[2][0] == "Q" or i[2][0] == "B"], 
            [coordinate_patch(i[2][3:5]) for i in list_epochs if i[2][0] == "Q" or i[2][0] == "B"], c= "palegoldenrod", lw = 2, zorder = 1)
    
    #Sets the colors to differentiate the types of between objects
    colors = ["palegreen", "turquoise"]

    #Plots the trajectory between objects
    for a in range(2) :
        current_list = [between_reward, between_unrewarded][a]
        if len(current_list) != 0 : #Line below : change the "between objects with/without reward" par bo rewarded or unrewarded
            plt.scatter([time_average[i[1]] for i in current_list], [coordinate_patch(i[2][3:5]) for i in current_list], c = colors[a], label = "between object " + ["with ", "without "][a] + "reward", marker= "x", zorder = 2)
        else : pass

    #List of the colors used to diffentiate the quarter turns
    colors = ["royalblue", "orangered", "orange", "gold", "chocolate", "slategray"]

    for a in range(6): #Plots the dot for each category of quarter turns
        current_list = [rewarded, extra, badDirection, wrongObject, doubleWrong, exploring][a]
        if len(current_list) != 0:
            plt.scatter([time_average[i[0]] for i in current_list], [coordinate_patch(i[2][3:5]) for i in current_list], c = colors[a], label = ["gogt", "extra", "gobt", "bogt", "bobt", "expl"][a], marker= "|", zorder = 3)
        else : pass
    if REMAINING_REWARDS : 
        for a in range(len(list_number_reward)) :
            plt.text(time_average[list_quarter_turn[list_number_reward[a][0]][0]] - 3, coordinate_patch(list_quarter_turn[list_number_reward[a][0]][2][3:5])-0.05 , s = str(list_number_reward[a][1]))

    plt.yticks(ticks = [1, 2, 3, 4], labels= ['NE', 'NW', 'SE', 'SW'])
    plt.xlabel('time (s)')
    plt.ylim(0,5)
    plt.legend(loc = 2)

    ############################################
    #Saves and closes the graph
    plt.title(codename)
    fig.savefig(mouseFolder_Path + '/' + session + '/' + session + '_ColoredDot.png',bbox_inches='tight')
    plt.close() #Avoids showing graphs 

############################################
    #Creates the graph that details the characteristics for the different types of trajectories

    #Plots the graph of analysis according to trajectory type
    nb_column = 5 #One for each type of trajectory
    nb_line = 4 #one for each graph to be plotted

    max_height_speed = []
    max_height_angular = []
    max_height_accel = []

    fig = plt.figure(figsize = (5 * nb_column, 5 * nb_line))
    #fig.suptitle("Analysis of different trajectory direction :" + session_direction + " session = " + session + '\n\n')
    for a in range(nb_column):
        current_movements = [list_quarter_turn, list_between_objects, list_toward_object, list_movement_not_quarter, list_of_stops][a]

    #Plots the trajectory of the current mod
        ax  = fig.add_subplot(nb_line, nb_column, 1 + a)
        for patch in collection_trapeze : #Plots the trapeze around the object
            for trapeze in collection_trapeze[patch] :
                shape = Polygon(np.array(collection_trapeze[patch][trapeze]), color = "lemonchiffon")
            #plt.gca().add_patch(shape)
                ax.add_patch(shape)

            for u in current_movements :#Plots each individual trajectory of the current category
                colors = plt.cm.rainbow(np.random.random())
                if len(u) != 0 :
                    ax.plot( xgauss[u[0] : u[1]], ygauss[u[0] : u[1]], linewidth = 0.5, c = colors )
                else : pass
    
    #Plots a colored dot at the begining and end of each epoch
        indices_start = [u[0] for u in current_movements]
        indices_end = [u[1] for u in current_movements]
        if len(indices_start) != 0 or len(indices_end)!= 0 :
            ax.scatter(xgauss[indices_start], ygauss[indices_start], linewidths= 0.1, color = "green")
            ax.scatter(xgauss[indices_end], ygauss[indices_end], linewidths= 0.1, color = "red")
        else : pass
        
        timeSpentIn = (sum([df.loc[df.index[epoch[1]], 'time'] - df.loc[df.index[epoch[0]], 'time'] for epoch in current_movements]), 2)

    #Sets the parameters of the graph
        ax.set_ylim(0, 500); ax.set_xlim(0, 500)
        ax.set_title(["Quarter turns : ", "Between objects : ", "Towards objects : ", "Other movements : ", "Stops : "][a] + str(len(current_movements)) + f"\n total time : {timeSpentIn}s")
        plt.xlim(12, 500); plt.ylim(12, 500)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    #Gets the speed and angular speed depending on the category
        speedy = [] ; angle_speedy = []; accelery = []
        for u in current_movements :
            for i in range(u[0], u[1] + 1) :
                speedy += [speed[i]]
                angle_speedy += [angular_speed[i]]
                accelery += [acceleration[i]]

    #Plots the distribution of speed for the type of trajectory
        if a != 4 :
            ax2 = fig.add_subplot(nb_line, nb_column, 1* nb_column +1 +a )
            if len(speedy)!= 0 :
                values, _ , _ = ax2.hist(speedy, bins = np.arange(0, 100, 1), density = True)
                max_height_speed.append( max(values) )
            else :
                ax2.hist([0], bins = np.arange(0, 100, 1), density = True)
                max_height_speed.append(0)
            ax2.set_xlabel("Speed (cm/s)")
            ax2.set_title('Speed')
            ax2.axvline(true_cut_speed, c = 'red', linewidth = 0.5)

    #Plots the angular speed for the type of trajectory
        if a != 4 :
            ax3 = fig.add_subplot(nb_line, nb_column, 2* nb_column +1 +a )
            if len(angle_speedy)!= 0 :
                values, _ , _ = ax3.hist(angle_speedy, bins = np.arange(-50, 50, .5), density = True)
                max_height_angular.append( max(values) )
            else  :
                ax3.hist([0], bins = np.arange(-50, 50, .5), density = True)
                max_height_angular.append(0)
            ax3.set_xlabel("Angular speed (degree/s)")
            ax3.text(-50, 0.02, "clockwise", ha = 'left') ; plt.text(50, 0.02, "counterclockwise", ha = 'right')
            ax3.set_title('Angular speed')
            #ax3.set_ylim(0, [0.07, 0.083, 0.18, 0.11, 0.55][a])
            ax3.axvline(15, c = 'red', linewidth = 0.5) ; plt.axvline(-15, c = 'red', linewidth = 0.5) ; plt.axvline(0, c= 'k', linewidth = 0.5)

    #Plots the acceleration for the type of trajectory
        if a != 4 :
            ax5 = fig.add_subplot(nb_line, nb_column, 3* nb_column +1 +a )
            if len(accelery)!= 0 :
                values, _ , _ = ax5.hist(accelery, bins = np.arange(-25, 25, 0.3), density = True)
                max_height_accel.append( max(values))
            else :
                ax5.hist([0], bins = np.arange(-25, 25, 0.3), density = True)
                max_height_accel.append(0)
            ax5.set_xlabel('Acceleration (cm.s-2)')
            ax5.set_title('Running acceleration')
    
    ############################################
    max_height_speed.sort() #Sorts the height of the different graphs of each type 
    max_height_angular.sort()
    max_height_accel.sort()

    for a in range(4) : #Sets the scale of all graphd of a category to the scale of the one with the biggest scale
        ax = plt.subplot(nb_line, nb_column, 1* nb_column +1 +a )
        ax.set_ylim(0, max_height_speed[-1] + 0.1* max_height_speed[-1]) #It's sorted in increasing order, so the last is the highest

        ax = plt.subplot(nb_line, nb_column, 2* nb_column +1 +a )
        ax.set_ylim(0, max_height_angular[-1] + 0.1* max_height_angular[-1])

        ax = plt.subplot(nb_line, nb_column, 3* nb_column +1 +a ) 
        ax.set_ylim(0, max_height_accel[-1] + 0.1* max_height_accel[-1])
        
    ############################################
    #Plots the time of stops after a rewarded quarter turn
    plt.subplot(nb_line, nb_column, nb_column * 2)
    variable = [time_average[u[1]] - time_average[u[0]]  for u in stops_type["rewarded"]]
    if len(variable) != 0 :
        plt.hist(variable, bins = np.arange(0, 2, 0.05))
    else : 
        plt.hist([0], bins = np.arange(0, 2, 0.05))
    plt.title("Stops duration after rewarded turn")
    
    ############################################
    #Plots the time of stops after a non-rewarded quarter turn
    plt.subplot(nb_line, nb_column, nb_column * 3)
    variable = [time_average[u[1]] - time_average[u[0]]  for u in stops_type["unrewarded"]]
    if len(variable) != 0 :
        plt.hist(variable, bins = np.arange(0, 2, 0.05))
    else : 
        plt.hist([0], bins = np.arange(0, 2, 0.05))
    plt.title("Stops duration after unrewarded turn")
    
    ############################################
    #Plots the time of stops after all quarter turns
    plt.subplot(nb_line, nb_column, nb_column * 4)
    variable = [time_average[u[1]] - time_average[u[0]]  for u in stops_type["unrewarded"]] + [time_average[u[1]] - time_average[u[0]]  for u in stops_type["rewarded"]]
    if len(variable) != 0 :
        plt.hist(variable, bins = np.arange(0, 2, 0.05))
    else : 
        plt.hist([0], bins = np.arange(0, 2, 0.05))
    plt.title("Stops duration")
    plt.tight_layout(rect=(0,0, 1, 0.98))

    ############################################
    #Saves and closes the figure
    plt.suptitle(codename)
    fig.savefig(mouseFolder_Path + '/' + session + '/' + session +  '_TrajectoriesAnalysis.png',bbox_inches='tight')
    plt.close()

    ############################################
    #Plots the graph which studies the quarter turns according to their direction

    clock_turn = [epoch for epoch in list_quarter_turn if epoch[2][1] == "w"] #Divides the quarter turns depending on their direction (CW or CCW)
    anti_clock_turn = [epoch for epoch in list_quarter_turn if epoch[2][1] == "k"]
    clock_angular_speed = [angular_speed[i] for u in clock_turn for i in range(u[0], u[1] + 1) ] 
    anti_angular_speed = [angular_speed[i] for u in anti_clock_turn for i in range(u[0], u[1] + 1)]
    clock_speed = [speed[i] for u in clock_turn for i in range(u[0], u[1] + 1)] ; anti_clock_speed = [speed[i] for u in anti_clock_turn for i in range(u[0], u[1] + 1)]
    
    ############################################
    #This is to create and save pickle for the different parameters in each session folder
    filename = mouseFolder_Path + '/' + session + '/' + 'pickle_clock_turn.pkl'
    with open(filename, 'wb') as f:
        pickle.dump(clock_turn, f)

    filename = mouseFolder_Path + '/' + session + '/' + 'pickle_clock_speed.pkl'
    with open(filename, 'wb') as f:
        pickle.dump(clock_speed, f)
    
    filename = mouseFolder_Path + '/' + session + '/' + 'pickle_anti_clock_speed.pkl'
    with open(filename, 'wb') as f:
        pickle.dump(anti_clock_speed, f)


    nb_line = 5 ; nb_column = 2
    fig = plt.figure( figsize= (10, 4 * nb_line))
    for a in range (nb_column):
        direction = ["clockwise" , "counterclockwise"][a]
        
############################################
#Plots the distribution of angular speed
        plt.subplot(nb_line, nb_column, 1 + a ) 
        if len([clock_angular_speed, anti_angular_speed][a]) != 0 :
            plt.hist([clock_angular_speed, anti_angular_speed][a], bins = np.arange(-50, 50, .5), density = True)
        else :
            plt.hist([0], bins = np.arange(-50, 50, .5), density = True)
        plt.xlabel("Angular speed (degree/s)")
        plt.ylim(0, 0.075)
        plt.title('Angular speed ' + direction)
        plt.text(-50, 0.02, "Clockwise", ha = 'left') ; plt.text(50, 0.02, "Counterclockwise", ha = 'right')
        plt.axvline(15, c = 'red', linewidth = 0.5) ; plt.axvline(-15, c = 'red', linewidth = 0.5) ; plt.axvline(0, c= 'k', linewidth = 0.5)
        plt.xlabel("Angular speed (degree/s)")

############################################
#Plots the distribution of speed 
        plt.subplot(nb_line, nb_column, 1* nb_column +1 + a )
        if len([clock_speed, anti_clock_speed][a])!= 0 :
            plt.hist([clock_speed, anti_clock_speed][a], bins = np.arange(0, 60, 1), density = True)
        else : 
            plt.hist([0], bins = np.arange(0, 60, 1), density = True)
        plt.xlabel("Speed (cm/s)")
        plt.ylim(0, 0.07)
        plt.title('Speed ' + direction)
        plt.axvline(true_cut_speed, c = 'red', linewidth = 0.5)
        
#################################
#Plots the individual speed profile of every quarter turn
        plt.subplot(nb_line, nb_column, nb_column*2 + 1 +a)
        for u in [clock_turn, anti_clock_turn][a] :
            plt.plot(time_average[u[0] : u[1] + 1]  - time_average[u[0]], speed[u[0] : u[1] + 1], linewidth = 0.5, c= "dimgray")
        plt.xlim(0, 0.7); plt.ylim(0, 60)

        xmed_clock, ymed_clock, x_clock,y_clock = [], [],[], []
        xmed_anti_clock, ymed_anti_clock, x_anti_clock,y_anti_clock = [], [],[], []
        if len(clock_turn) != 0:
            xmed_clock, ymed_clock = compute_median_trajectory([speed[u[0] : u[1] + 1] for u in clock_turn],
                                                       [time_average[u[0] : u[1] + 1] for u in clock_turn])

            x_clock,y_clock = ([speed[u[0] : u[1] + 1] for u in clock_turn],
                                    [time_average[u[0] : u[1] + 1] for u in clock_turn])

        if len(anti_clock_turn) != 0:
            xmed_anti_clock, ymed_anti_clock = compute_median_trajectory([speed[u[0] : u[1] + 1] for u in anti_clock_turn],
                                                                 [time_average[u[0] : u[1] + 1] for u in anti_clock_turn])
            x_anti_clock,y_anti_clock = ([speed[u[0] : u[1] + 1] for u in anti_clock_turn],
                                    [time_average[u[0] : u[1] + 1] for u in anti_clock_turn])

        # Save both clock and anti-clock trajectories in pickle file
        filename = mouseFolder_Path + '/' + session + '/' + 'pickle_median_trajectory.pkl'
        with open(filename, 'wb') as f:
            pickle.dump((xmed_clock, ymed_clock, xmed_anti_clock, ymed_anti_clock), f)

        filename = mouseFolder_Path + '/' + session + '/' + 'pickle_trajectory.pkl'
        with open(filename, 'wb') as f:
            pickle.dump((x_clock,y_clock, x_anti_clock,y_anti_clock), f)

        # Adaptation for plotting
        if len([clock_turn, anti_clock_turn][a]) != 0:
            xmed, ymed = compute_median_trajectory([speed[u[0] : u[1] + 1] for u in [clock_turn, anti_clock_turn][a]],
                                           [time_average[u[0] : u[1] + 1] for u in [clock_turn, anti_clock_turn][a]])
            plt.plot(xmed, ymed, c= 'crimson'); plt.title("speed of the quarters turns " + direction)


        plt.ylabel("speed in cm/s"); plt.xlabel("time in s")
        
#################################
#Plots the individual profile of the direction changes of every quarter turn
        plt.subplot(nb_line, nb_column, nb_column * 3 + a+ 1)
        list_temp_orientation = []
        for u in [clock_turn, anti_clock_turn][a] :
            temp_orientation = angles[u[0] : u[1] + 1] - angles[u[0]]
            for i in range(1, len(temp_orientation)):
                if (temp_orientation[i] - temp_orientation[i - 1]) < -200 :#If the orientation passed the threshold and moved to the other side, moves it to stay within the continuity
                    temp_orientation[ i ] += 360
                elif (temp_orientation[i] - temp_orientation[i - 1]) > 200 :
                    temp_orientation[i] -= 360
            list_temp_orientation.append(temp_orientation)
            plt.plot(time_average[u[0]: u[1] + 1] - time_average[u[0]], temp_orientation, linewidth = 0.5, c= "dimgray")
        plt.ylim(-180, 180); plt.xlim(0, 0.7)
        plt.ylabel("changes in direction (degree)"); plt.xlabel("time in s")
        if len([clock_turn, anti_clock_turn][a]) != 0:
            xmed, ymed = compute_median_trajectory(list_temp_orientation, [time_average[u[0] : u[1] + 1] for u in [clock_turn, anti_clock_turn][a]])
            plt.plot(xmed, ymed, c= 'crimson')
        plt.title("direction changes " + direction)

############################################
#Computes the cumulative sums in accordance to time
    clockcum = np.cumsum([1 if  indice in [u[0] for u in clock_turn] else 0 for indice in range(len(time_average))]) 
    anticum = np.cumsum([1 if  indice in [u[0] for u in anti_clock_turn] else 0 for indice in range(len(time_average))])
    reward_time = np.cumsum([1 if  indice in [u[0] for u in rewarded] else 0 for indice in range(len(time_average))])
    unrewarded_time = np.cumsum([1 if  indice in [u[0] for u in unrewarded] else 0 for indice in range(len(time_average))])

    #Plots the cumulative sum of each direction of quarter turn
    plt.subplot(nb_line, nb_column, nb_column * 4 + 1) 
    plt.plot(time_average, clockcum, c= "moccasin", label = "Clockwise")
    plt. plot(time_average, anticum, c= "powderblue", label = "Counterclockwise")
    plt.title("Cumulative number of quarter turns according to direction")
    plt.legend()
    plt.ylabel("Number of quarter turns"); plt.xlabel("Time (s)")

############################################
    #Plots the cumulative number of rewarded and unrewarded quarter turns
    plt.subplot(nb_line, nb_column, nb_column * 4 + 2)
    plt.plot(time_average, reward_time, c= "royalblue", label = "Rewarded")
    plt. plot(time_average, unrewarded_time, c= "sandybrown", label = "Unrewarded")
    plt.legend()
    plt.title("Was the quarter turn rewarded ?")
    plt.ylabel("Number of quarter turns"); plt.xlabel("Time (s)")
    plt.tight_layout()

############################################
    #Saves and closes the figure
    fig.savefig(mouseFolder_Path + '/' + session + '/' + session +  '_QuarterDirection.png',bbox_inches='tight')
    plt.close()

############################################
    #Plots the angular speed for all movements
    plt.figure(figsize  = (5, 5))
    angle_speedy = [angular_speed[i]  for u in list_epochs for i in range(u[0], u[1] + 1)]
    plt.hist(angle_speedy, bins = np.arange(-50, 50, .5), density = True)
    plt.title("angular speed")

############################################
    #Saves and closes the graph
    plt.savefig(mouseFolder_Path + '/' + session + '/' + session +  '_AngularSpeed.png', bbox_inches='tight')
    plt.close()

########################################################################################
    return 0 #Indicates that the function finished without problem
    ########################################################################################

def phaseNumberDirectionAndCNO(path) :
    try :
        session_infos = pd.read_csv(path)
        
        if not session_infos.loc[session_infos.index[0], "allowRewardDelivery"] : phase =  0 #If no reward can be given, then it's the free exploration
        elif session_infos.loc[session_infos.index[0], "number_of_alternativeObject"] == 1 : phase =  4 #If only one alternative is available for the objects at a given time in the session, it's session 4
        elif session_infos.loc[session_infos.index[0], "number_of_alternativeObject"] == 3 : phase = 3 #Then it's phase 3
        elif session_infos.loc[session_infos.index[0], "potentialRewardedDirections"] == '[90, 270]' : phase = 1
        else : phase = 2

    except : 
        return -1, "[]", False
    
    try : #If the error is here, then the missing part is the cno which means it cannot have happened
        return phase, session_infos.loc[session_infos.index[0], "potentialRewardedDirections"], session_infos.loc[session_infos.index[0], "injectionCNO"] != "none"
    except : 
        return phase, session_infos.loc[session_infos.index[0], "potentialRewardedDirections"], False

In [96]:
#Old cell with functions for the previous first draft of analysis
#Function to process the trajectories and speed 
def trajectories_and_speed(session):
    csvCentroid_fullpath=Folder_path + mouseName + '/' + session + '/' + session + '_centroidTXY.csv'
    if not os.path.isfile(csvCentroid_fullpath):
        return - 1
    else:
        df=pd.read_csv(csvCentroid_fullpath)
        df=df.dropna()
        df=df.loc[df['time'] >15]
        df=df[df['xposition'].between(1,500) & df['yposition'].between(1,500)]
    
        xpositions=df['xposition'].to_numpy()
        ypositions=df['yposition'].to_numpy()
        
        distances=np.array([((((xpositions[i]-xpositions[i-1])**2)+((ypositions[i]-ypositions[i-1])**2))**0.5) for i in range(1,len(ypositions))])
        resolution=resolution = 512,512

        df['yposition'] = resolution[1] - df['yposition']

        distances = distances *(0.84/resolution[0]) #Converts distance to m with apparatus length = 84 cm
        totaldistance=np.sum(distances)
        averagerunningspeed=np.sum(distances)/(12*60)
        timeofframes=df['time'].to_numpy()
        timebeweenframe=np.diff(timeofframes)
        speeds=np.divide(distances,timebeweenframe)

        fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(5*2, 5*1),
                                constrained_layout=True)

        df.plot(x='xposition',y='yposition',linewidth=0.5,ax=axs[0],c='k')
        axs[0].set_xlim(0,500)
        axs[0].set_ylim(0,500)
        axs[0].set_title('Total distance run=' + np.array2string(totaldistance,precision=2) + 'm')
        axs[0].get_legend().remove()
        axs[0].axis('off')

        nEvent, binedges, patches = axs[1].hist(speeds*100, bins=np.arange(0, 100, 100/25), density=True, facecolor='k', alpha=0.75)
        bincenters = 0.5*(binedges[1:]+binedges[:-1])

        cruisespeedbinsindexes=np.where(bincenters>10) #10cm/s is the arbitrary limit between immobility and running
        cruiseSpeedBinValues=np.take(bincenters,cruisespeedbinsindexes)
        cruiseSpeedBinsNevent=np.take(nEvent,cruisespeedbinsindexes)
        meanCruiseSpeed=np.sum(np.multiply(cruiseSpeedBinValues,cruiseSpeedBinsNevent))/np.sum(cruiseSpeedBinsNevent)

        immobilitybinsindexes=np.where(bincenters<10)
        ratioRunvsImmob=np.divide(np.sum(np.take(nEvent,cruisespeedbinsindexes)),np.sum(np.take(nEvent,immobilitybinsindexes)))
        #print(ratioRunvsImmob)

        axs[1].set_title('Average cruise speed=' + np.array2string(meanCruiseSpeed,precision=2) + 'cm/s      Ratio Run/Immo=' + np.array2string(ratioRunvsImmob,precision=2))
        #axs[1].spines[['right', 'top']].set_visible(False) #Need to comment if there is an error 
        axs[1].set_xlabel('Speed (cm/s)')
        axs[1].set_ylabel('Proba of event')
        axs[1].set_ylim([0, 0.1])
        plt.savefig(Folder_path + mouseName + '/' + session + '/' + session + '_TrajectoriesAndSpeed.png',bbox_inches='tight')
        plt.close() #Avoids showing graphs 
    return 0

#Function to process the type of turns
def NumberOfQTurns(session):
    csvTurnFile_fullpath=Folder_path + mouseName + '/' + session + '/' + session + '_turnsinfo.csv'
    if not os.path.isfile(csvTurnFile_fullpath):
        #print(f"The turn file from {session} was not found")
        return -1 #Signals an error
    else:
        df=pd.read_csv(csvTurnFile_fullpath)
        df=df.loc[df['time'] >15] #Gets rid of the first 15sec because tracking is usually bad at this moment
        #display(df)
        typeofturns=np.unique(df['typeOfTurn'].to_numpy())
        #print('types of turn found in this session:') #Shows all types of turns that can be found in this session, printed under the table 
        #for thistypeofturn in typeofturns:
            #print(thistypeofturn)
        fig, ax = plt.subplots(figsize=(5, 5))

        lasturn_time=df['time'].iloc[-1]
        binsize=10
        bins=np.arange(0, lasturn_time+binsize,binsize)

        allturns_time=df['time'].to_numpy()
        n, bins, patches = ax.hist(allturns_time, bins, density=False, histtype='step', cumulative=True, label='trapeze changes',color=turnPalette['all turns'])
        #ax.plot(bins[:-1],n,label='all turns',color=turnPalette['all turns'][0],linestyle=turnPalette['all turns'][1])
        turn_times=df.loc[df['Rewarded']]
        if len(turn_times) == 0 :
            turn_times=df.loc[df['typeOfTurn']=="gogt"]
        turn_times=turn_times['time'].to_numpy()
        n, bins, patches = ax.hist(turn_times, bins, density=False, histtype='step', cumulative=True, label="rewarded",color=turnPalette["gogd"])
          
        ax.legend(loc='upper left')
        ax.set_ylim([0, 800])
        plt.savefig(Folder_path + mouseName + '/' + session + '/' + session + '_NbOfQTurns.png',bbox_inches='tight')
        #pngfullpath=Folder_path + mouseName + '/' + session + '/' + session + '_NbOfQTurns.png'
        plt.close()

    return 0 #Signals no problem

#Function to create the empty HTML file
header1="<!DOCTYPE html>\n<html>\n<head>\n<meta charset='utf-8' />\n<title>\n"
header2="</title>\n</head><body><p>\n"
bottom="</p></body></html>"

#Ecrire les noms de sessions sous la forme d'un titre HTML pour eviter d'avoir a append des strings dans images[] a la fin !!!

def create_empty_html(path,name): 
        if not path.endswith(".html"):
            path=path+".html"
        with open(path,"w") as f:
            f.write(header1+name+header2)
            f.write(bottom) 
            f.close()

#Function to insert figures into the HTML
def insert_in_html(path,insertList,name):
    bottom="</p></body></html>"

    if not os.path.exists(path):
        if name is None:
            name=os.path.basename(path)
        create_empty_html(path,name)
    if isinstance(insertList,str): 
        insertList=insertList.split("\n")
        
    with open(path,"w") as f:
        #contents=[line.rstrip("\n") for line in f if line!="\n"]
        #f.seek(0)
        #f.truncate() 
        #contents=contents[:-1]+insertList+[bottom]
        contents = insertList+[bottom]
        f.write("\n".join(contents))
        f.close()
        
def remove_lines_in_html(path,contents):
    if not os.path.exists(path):
        return
    newLines=[]
    with open(path,"r+") as f:
        for line in f:
            if (contents not in line) and (line!="\n"):
                newLines.append(line.strip("\n"))
        f.seek(0)
        f.truncate()
        f.write("\n".join(newLines))
        f.close()

#Function to create a progress bar
def update_progress(progress, barLength=50):
    def _statusupdate(progress):
        status = ""
        if isinstance(progress, int):
            progress = float(progress)
        if not isinstance(progress, float):
            progress = 0
            status = "Error: progress var must be float\r\n"
        if progress < 0:
            progress = 0
            status = "Something went WRONG...\r\n"
        if progress < 1 and progress >= 0:
            status = "Computing..."
        if progress >= 0.999:
            progress = 1
            status = 'Done ✓      '
        return status

    text = "\r" + "Progress: [{0}] {1}% {2}".format(" " * int(round(barLength * progress)) + '---{,,">' +
        " " * (barLength-int(round(barLength * progress))), int(round(progress * 100)), _statusupdate(progress))

    sys.stdout.write(text)
    sys.stdout.flush()

## Run the analyzes and create an HTML 

In [97]:
#for i in range(len(sessionFolders)): #Need to find a way to print when a file is not found without disturbing the progress bar : appear at the end of progression
#    if not os.path.isfile(csvCentroid_fullpath):
#        print(f"Centroid file from {sessionFolders[i]} was not found")
#    elif not os.path.isfile(csvTurnFile_fullpath):
#        print(f"The turn file from {sessionFolders[i]} was not found")

#HTML for the animal 

generalName=mouseName+"_Analysis"

generalFolder=os.path.join(mouseFolder_Path,"Analysis") #Ajoute un dossier appele "Analysis" dans le dossier de la souris
if not os.path.exists(generalFolder):
    os.mkdir(generalFolder)
generalPath=os.path.join(generalFolder,generalName+".html") #Ajoute le fichier HTML dans le dossier
secondPath = os.path.join(generalFolder,mouseName+ "_Behavior" +".html")
if not os.path.exists(generalFolder + "/ListSessionsAnalyzed.txt"):
    listSessionAnalyzed = []
    with open(generalFolder + "/ListSessionsAnalyzed.txt", "w") as f :
        f.close()
else : 
    with open(generalFolder + "/ListSessionsAnalyzed.txt", "r") as f :
        listSession = f.readlines()
        listSessionAnalyzed = [session[0:-1] if session[-1]=="\n" else session for session in listSession]

#Cell to test Alice's analysis
for i in range (len(sessionFolders)):
    print(i + 1)
    csvCentroid_fullpath=Folder_path + mouseName + '/' + sessionFolders[i] + '/' + sessionFolders[i] + '_centroidTXY.csv'
    csvTurnFile_fullpath=Folder_path + mouseName + '/' + sessionFolders[i] + '/' + sessionFolders[i] + '_turnsinfo.csv'
    if sessionFolders[i] not in listSessionAnalyzed :
        AnalysisFunction(sessionFolders[i], mouseFolder_Path) #Creates the graph for analysis of the types of trajectory, analysis of quarter turnS depending on their direction and Behavior
        NumberOfQTurns(sessionFolders[i]) #Creates the cumulative sum of turns (third graph of the Analysis HTML)
        trajectories_and_speed(sessionFolders[i]) #Creates the two first graph of the Analysis HTML

        clear_output() #Clears everything so the memory is not overused
        plt.close('all')
        gc.collect() #Running the default Python Garbage Collector
    #update_progress(i/(len(sessionFolders)-1))

#Creates HTML image tag
with open(generalFolder + "/ListSessionsAnalyzed.txt", "w") as f : #File is created above if it was not
    for i in sessionFolders:
        f.write(i + "\n")
    f.close()

images=[header1+mouseName+header2]
for i in range(len(sessionFolders)):

    base_path = mouseFolder_Path + '/' + sessionFolders[i] + '/' + sessionFolders[i]
    
    phase, direction, cno = phaseNumberDirectionAndCNO(base_path + "_sessionparam.csv")
    dataSouris = "Phase " + str(phase) + " Session " + str(i + 1)
    if type(direction) != type("") : direction = "[]"
    if phase != 0 : dataSouris = dataSouris + " direction : " + direction
    if cno : dataSouris = dataSouris + " session CNO"

    #base_path = "NETDATA/EqpRobbe/Lucie/Foraging_task/Datas_foraging" + mouseName + '/' + sessionFolders[i] + '/' + sessionFolders[i]

    trajectorypng = base_path + '_TrajectoriesAndSpeed.png'
    speedpng = base_path + '_NbOfQTurns.png'
    angularpng = base_path + '_AngularSpeed.png'
    trajectorypngfullanalysis= base_path + '_TrajectoriesAnalysis.png'
    turnspngfullpath= base_path + '_QuarterDirection.png'
    images.append(f'<h3 style="text-align: center;"><span style="text-decoration: underline;"><strong>{sessionFolders[i]}</strong></span></h3>')
    images.append(f'<h4 style="text-align: center;">{dataSouris}</h4>')
    images.append(f'<p><img src="{trajectorypng}" alt="" /><img src="{speedpng}" alt="" /><img src="{angularpng}" alt="" /></p>')
    images.append(f'<p><img src="{trajectorypngfullanalysis}" alt="" /><img src="{turnspngfullpath}" alt="" /></p>')
    #images.append(f"<a href=#{sessionFolders[i]}><img src='{trajectorypngfullanalysis}' alt='{sessionFolders[i]}' title='{sessionFolders[i]}'/></a>")
    #images.append(f"<a href=#{sessionFolders[i]}><img src='{turnspngfullpath}' alt='{sessionFolders[i]}' title='{sessionFolders[i]}'/></a>")
    images.append('<p></p>')
    #remove_lines_in_html(generalPath,Folder_path)
insert_in_html(generalPath, images, mouseName)

#Creates the HTML for the graph showing the behavior of the animal
with open(generalFolder + "/ListSessionsAnalyzed.txt", "w") as f : #File is created above if it was not
    for i in sessionFolders:
        f.write(i + "\n")
    f.close()

colored_image = [header1+mouseName+header2]
for i in range(len(sessionFolders)):

    base_path = mouseFolder_Path + '/' + sessionFolders[i] + '/' + sessionFolders[i]
    
    phase, direction, cno = phaseNumberDirectionAndCNO(base_path + "_sessionparam.csv")
    dataSouris = "Phase " + str(phase) + " Session " + str(i + 1)
    if type(direction) != type("") : direction = "[]"
    if phase != 0 : dataSouris = dataSouris + " direction : " + direction
    if cno : dataSouris = dataSouris + " session CNO"
    
    coloredpath= base_path + '_ColoredDot.png'
    colored_image.append(f'<h3 style="text-align: center;"><span style="text-decoration: underline;"><strong>{sessionFolders[i]}</strong></span></h3>')
    colored_image.append(f'<h4 style="text-align: center;">{dataSouris}</h4>')
    colored_image.append(f'<p><img src="{coloredpath}" alt="" /></p>')
    colored_image.append('<p></p>')

insert_in_html(secondPath, colored_image, mouseName)

1


In [ ]:
#Cell to do some tests

#images=[]
#     for i in range(len(sessionFolders)):
#     trajectorypngfullpath=Folder_path + mouseName + '/' + sessionFolders[i] + '/' + sessionFolders[i] + '_TrajectoriesAndSpeed.png'
#     turnspngfullpath=Folder_path + mouseName + '/' + sessionFolders[i] + '/' + sessionFolders[i] + '_NbOfQTurns.png'
#     #images.append(sessionFolders[i]) --> a modifier pour faire apparaitre comme un titre (version HTML), comme ca il n'est plus dans la boucle et ca evite l'accumulation a chaque fois 
#     #que l'on run le code
#     images.append(f"<a href=#{sessionFolders[i]}><img src='{trajectorypngfullpath}' alt='{sessionFolders[i]}' title='{sessionFolders[i]}'/></a>")
#     images.append(f"<a href=#{sessionFolders[i]}><img src='{turnspngfullpath}' alt='{sessionFolders[i]}' title='{sessionFolders[i]}'/></a>")
#remove_lines_in_html(generalPath,Folder_path)
# #Ici : si image[i] est deja inseree dans l'HTML, ne pas la rajouter a nouveau 
#insert_in_html(generalPath, images, mouseName)

In [ ]:
#session_infos = pd.read_csv(base_path + "_sessionparam.csv")
#infos = list(session_infos.iloc[0])
#infos[3] != "none"